# Instalações das bibliotecas

In [1]:
!pip install python-telegram-bot==13.7
!pip install llama_cpp_python
!pip install pytelegrambotapi

# LLama (Inteligência Artificial)

Configuração da IA

In [3]:
from llama_cpp import Llama
from llama_cpp.llama_tokenizer import LlamaHFTokenizer

llm = Llama.from_pretrained(
    repo_id="meetkai/functionary-small-v2.4-GGUF",
    filename="functionary-small-v2.4.Q4_0.gguf",
    chat_format="functionary-v2",
    tokenizer=LlamaHFTokenizer.from_pretrained("meetkai/functionary-small-v2.4-GGUF"),
    n_gpu_layers=-1,
)

c:\Users\danie\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
llama_model_loader: loaded meta data with 25 key-value pairs and 291 tensors from C:\Users\danie\.cache\huggingface\hub\models--meetkai--functionary-small-v2.4-GGUF\snapshots\a0d171eb78e02a58858c464e278234afbcf85c5c\.\functionary-small-v2.4.Q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:            

# Funções do Chatbot

Definindo as funções

In [27]:
import sqlite3

# Funções
def greet_user():
    return "Olá! Como posso ajudar você hoje?"

def search_books(name):
    conn = sqlite3.connect('bookstore.db', timeout=10)
    cursor = conn.cursor()
    
    query = "SELECT title FROM books WHERE title LIKE ?"
    cursor.execute(query, ('%' + name + '%',))
    books = cursor.fetchall()
    
    conn.close()
    
    if books:
        return "Livros encontrados:\n" + "\n".join(f"* {book[0]}" for book in books)
    else:
        return "Nenhum livro encontrado com esse nome."

# Estrutura para armazenar o carrinho
cart = []

def add_to_cart(book_title):
    conn = sqlite3.connect('bookstore.db', timeout=10)
    cursor = conn.cursor()
    
    query = "SELECT title, price FROM books WHERE title = ?"
    cursor.execute(query, (book_title,))
    book = cursor.fetchone()
    
    conn.close()
    
    if book:
        cart.append(book)
        return f"{book_title} foi adicionado ao seu carrinho."
    else:
        return "Livro não encontrado."

def remove_from_cart(book_title):
    for book in cart:
        if book[0] == book_title:
            cart.remove(book)
            return f"{book_title} foi removido do seu carrinho."
    return "Livro não encontrado no seu carrinho."

def view_cart():
    if cart:
        total = sum(book[1] for book in cart)
        books_in_cart = "\n".join(f"* {book[0]} - R${book[1]:.2f}" for book in cart)
        return f"Livros no seu carrinho:\n{books_in_cart}\n\nTotal: R${total:.2f}"
    else:
        return "Seu carrinho está vazio."

def checkout():
    if cart:
        total = sum(book[1] for book in cart)
        cart.clear()  # Esvaziar o carrinho após a compra
        return f"Sua compra foi finalizada com sucesso. Total: R${total:.2f}"
    else:
        return "Seu carrinho está vazio."

def get_author_by_book_title(book_title):
    conn = sqlite3.connect('bookstore.db', timeout=10)
    cursor = conn.cursor()
    
    query = '''
        SELECT authors.name 
        FROM books 
        JOIN authors ON books.author_id = authors.id 
        WHERE books.title = ?
    '''
    cursor.execute(query, (book_title,))
    author = cursor.fetchone()
    
    conn.close()
    
    if author:
        return f"O autor do livro '{book_title}' é {author[0]}."
    else:
        return "Autor não encontrado para esse livro."

# Mapeamento das funções
functions_map = {
    "greet_user": greet_user,
    "search_books": search_books,
    "add_to_cart": add_to_cart,
    "remove_from_cart": remove_from_cart,
    "view_cart": view_cart,
    "checkout": checkout,
    "get_author_by_book_title": get_author_by_book_title,
}

# Ferramentas do chatbot
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_books",
            "description": "Search for a specific book",
            "parameters": {
                "type": "object",
                "properties": {"name": {"type": "string", "description": "The name of the book"}},
                "required": ["name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "greet_user",
            "description": "Greet the user and ask how to help",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "add_to_cart",
            "description": "Add a book to the cart",
            "parameters": {
                "type": "object",
                "properties": {
                    "book_title": {"type": "string", "description": "The title of the book to add"},
                },
                "required": ["book_title"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "remove_from_cart",
            "description": "Remove a book from the cart",
            "parameters": {
                "type": "object",
                "properties": {
                    "book_title": {"type": "string", "description": "The title of the book to remove"},
                },
                "required": ["book_title"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "view_cart",
            "description": "View the prices of books in the cart and the total amount",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "checkout",
            "description": "Checkout and finalize the purchase",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_author_by_book_title",
            "description": "Get the author of a specific book",
            "parameters": {
                "type": "object",
                "properties": {
                    "book_title": {"type": "string", "description": "The title of the book"},
                },
                "required": ["book_title"],
            },
        },
    },
]


# Banco SQLite

Configuração do Banco SQLite

In [28]:
import sqlite3

conn = sqlite3.connect('bookstore.db', timeout=10)
cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS books')
cursor.execute('DROP TABLE IF EXISTS authors')

# Criando a tabela authors
cursor.execute('''
CREATE TABLE IF NOT EXISTS authors (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL
)
''')

# Criando a tabela books
cursor.execute('''
CREATE TABLE IF NOT EXISTS books (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author_id INTEGER,
    price REAL,
    FOREIGN KEY (author_id) REFERENCES authors (id)
)
''')

# Inserindo autores
authors = [('J.K. Rowling',), ('J.R.R. Tolkien',), ('George Orwell',)]
cursor.executemany('INSERT INTO authors (name) VALUES (?)', authors)

# Inserindo livros
books = [
    ('Harry Potter and the Philosopher\'s Stone', 1, 19.99),
    ('Harry Potter and the Chamber of Secrets', 1, 24.99),
    ('The Hobbit', 2, 15.99),
    ('1984', 3, 12.99),
]
cursor.executemany('INSERT INTO books (title, author_id, price) VALUES (?, ?, ?)', books)

# Salvando (commit) as alterações e fechando a conexão
conn.commit()
conn.close()

# Chatbot

Configuração do Chatbot

In [29]:
import telebot
import json

# Altere para o token do seu bot
API_TOKEN = '7112544250:AAF7irmOrLAo92-Sb-jqAptH7zJM7pnBYLo'

bot = telebot.TeleBot(API_TOKEN)

def get_function_details(llm, message, tools):
    result = llm.create_chat_completion(
        messages=[{"role": "user", "content": message}],
        tools=tools,
        tool_choice="auto",
    )
    function_call = result["choices"][0]["message"]["function_call"]
    function_name = function_call["name"]
    arguments = json.loads(function_call["arguments"])
    return function_name, arguments

def execute_function(function_name, arguments, functions_map):
    function = functions_map.get(function_name)
    if function:
        return function(**arguments)
    return f"Deculpe, eu não entendi o que você quis dizer."

@bot.message_handler(func=lambda message: True)
def reply_to_message(message):
    function_name, arguments = get_function_details(llm, message.text, tools)
    response = execute_function(function_name, arguments, functions_map)
    bot.reply_to(message, response)

Ativação do Chatbot

In [30]:

bot.polling()

Llama.generate: prefix-match hit

llama_print_timings:        load time =   89063.21 ms
llama_print_timings:      sample time =       3.11 ms /    11 runs   (    0.28 ms per token,  3538.12 tokens per second)
llama_print_timings: prompt eval time =   22934.22 ms /   120 tokens (  191.12 ms per token,     5.23 tokens per second)
llama_print_timings:        eval time =    4077.69 ms /    10 runs   (  407.77 ms per token,     2.45 tokens per second)
llama_print_timings:       total time =   27088.15 ms /   130 tokens
from_string grammar:
book-title-kv ::= ["] [b] [o] [o] [k] [_] [t] [i] [t] [l] [e] ["] space [:] space string 
space ::= space_6 
string ::= ["] string_7 ["] space 
char ::= [^"\] | [\] char_4 
char_4 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
root ::= [{] space book-title-kv [}] space 
space_6 ::= [ ] | 
string_7 ::= char string_7 | 

Llama.generate: prefix-match hit


book-title-kv ::= "\"book_title\"" space ":" space string
char ::= [^"\\] | "\\" (["\\/bfnrt] | "u" [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F])
root ::= "{" space book-title-kv "}" space
space ::= " "?
string ::= "\"" char* "\"" space



llama_print_timings:        load time =   89063.21 ms
llama_print_timings:      sample time =     124.02 ms /    13 runs   (    9.54 ms per token,   104.82 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    5375.38 ms /    13 runs   (  413.49 ms per token,     2.42 tokens per second)
llama_print_timings:       total time =    5599.52 ms /    14 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   89063.21 ms
llama_print_timings:      sample time =       0.28 ms /     1 runs   (    0.28 ms per token,  3546.10 tokens per second)
llama_print_timings: prompt eval time =    2213.24 ms /    11 tokens (  201.20 ms per token,     4.97 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =    2219.94 ms /    12 